In [ ]:
!pip install --upgrade plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 53.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from pyspark.sql.functions import * 
from pyspark.sql.types import *

In [ ]:
df = spark.read.csv('car_price_prediction.csv', header=True)
df.show(5)

+--------+-----+----+------------+-------+----------+---------+----------------+---------+-------------+---------+---------+-------------+------------+------+----------------+------+-------+
|      ID|Price|Levy|Manufacturer|  Model|Prod. year| Category|Leather interior|Fuel type|Engine volume|  Mileage|Cylinders|Gear box type|Drive wheels| Doors|           Wheel| Color|Airbags|
+--------+-----+----+------------+-------+----------+---------+----------------+---------+-------------+---------+---------+-------------+------------+------+----------------+------+-------+
|45654403|13328|1399|       LEXUS| RX 450|      2010|     Jeep|             Yes|   Hybrid|          3.5|186005 km|      6.0|    Automatic|         4x4|04-May|      Left wheel|Silver|     12|
|44731507|16621|1018|   CHEVROLET|Equinox|      2011|     Jeep|              No|   Petrol|            3|192000 km|      6.0|    Tiptronic|         4x4|04-May|      Left wheel| Black|      8|
|45774419| 8467|   -|       HONDA|    FIT|   

In [ ]:
#dataframe structure
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Levy: string (nullable = true)
 |-- Manufacturer: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Prod. year: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Leather interior: string (nullable = true)
 |-- Fuel type: string (nullable = true)
 |-- Engine volume: string (nullable = true)
 |-- Mileage: string (nullable = true)
 |-- Cylinders: string (nullable = true)
 |-- Gear box type: string (nullable = true)
 |-- Drive wheels: string (nullable = true)
 |-- Doors: string (nullable = true)
 |-- Wheel: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Airbags: string (nullable = true)



In [ ]:
#Renaming Columns
df = df.withColumnRenamed('Leather interior','Leather_interior')
df = df.withColumnRenamed('Prod. year','Prod_year')

In [ ]:
#Number of duplicate data in dataset
df.groupBy("Manufacturer","Model","Mileage","Price")\
.count()\
.filter("count > 1")\
.sort("count",ascending=False)\
.show(50)

+-------------+------+---------+-----+-----+
| Manufacturer| Model|  Mileage|Price|count|
+-------------+------+---------+-----+-----+
|        HONDA|   FIT|     0 km|   30|   30|
|       TOYOTA| Camry| 35058 km|18817|   24|
|       TOYOTA| Camry|314373 km|  392|   23|
|       TOYOTA| Camry|179381 km|11133|   23|
|       TOYOTA| Camry|156518 km| 3607|   22|
|       TOYOTA| Camry|190549 km| 5018|   22|
|       TOYOTA| Camry|102397 km|  784|   22|
|MERCEDES-BENZ| E 300| 26802 km|  314|   22|
|       TOYOTA| Camry|398069 km| 1098|   21|
|       TOYOTA| Camry|130478 km|  862|   20|
|       TOYOTA| Camry|159379 km| 3136|   20|
|MERCEDES-BENZ| E 350|189530 km|12388|   19|
|       TOYOTA| Camry|109798 km| 5488|   18|
|MERCEDES-BENZ| E 350|219030 km|29793|   17|
|MERCEDES-BENZ| E 350|119379 km|16621|   17|
|        LEXUS|GX 460|275240 km|15681|   17|
|MERCEDES-BENZ| E 350|274771 km|  314|   17|
|       TOYOTA| Camry|225510 km|  314|   17|
|MERCEDES-BENZ| E 350|229904 km|  470|   17|
|       TO

In [ ]:
#removing Duplicate values
df = df.dropDuplicates()

In [ ]:
df = df.withColumn("Mileage_km",regexp_replace("MIleage","[a-z]","").alias("Mileage_km").cast('int'))

In [ ]:
#Dropping a column
df = df.drop("ID","Mileage")

# Different Manufacturer order by alphabatic order
 Note: sort() will sort the value in individual partition whereas orderBy does it in sigle one.

 

In [ ]:
df.select("Manufacturer").distinct().orderBy("Manufacturer").show(10)

+------------+
|Manufacturer|
+------------+
|       ACURA|
|  ALFA ROMEO|
|ASTON MARTIN|
|        AUDI|
|     BENTLEY|
|         BMW|
|       BUICK|
|    CADILLAC|
|   CHEVROLET|
|    CHRYSLER|
+------------+
only showing top 10 rows



In [ ]:
#top 10 manufacturer by count
top_10_manufacturer = df.groupBy("Manufacturer").count().orderBy("count",ascending=False)
top_10_manufacturer.show()

+-------------+-----+
| Manufacturer|count|
+-------------+-----+
|      HYUNDAI| 3729|
|       TOYOTA| 3606|
|MERCEDES-BENZ| 2043|
|         FORD| 1088|
|    CHEVROLET| 1047|
|          BMW| 1036|
|        HONDA|  960|
|        LEXUS|  927|
|       NISSAN|  645|
|   VOLKSWAGEN|  571|
|    SSANGYONG|  439|
|          KIA|  417|
|         OPEL|  396|
|   MITSUBISHI|  288|
|       SUBARU|  274|
|         AUDI|  254|
|        MAZDA|  175|
|         JEEP|  138|
|       DAEWOO|   91|
|        DODGE|   90|
+-------------+-----+
only showing top 20 rows



Count of different model each Manufacturer have in the dataset


In [ ]:
#total number of cars dataset consists of for each model
df.groupBy("Manufacturer","Model").count().orderBy("count",ascending=False).show(10)

+-------------+--------+-----+
| Manufacturer|   Model|count|
+-------------+--------+-----+
|       TOYOTA|   Prius| 1069|
|      HYUNDAI|  Sonata| 1067|
|       TOYOTA|   Camry|  929|
|      HYUNDAI| Elantra|  910|
|MERCEDES-BENZ|   E 350|  534|
|      HYUNDAI|Santa FE|  527|
|        HONDA|     FIT|  444|
|      HYUNDAI|      H1|  435|
|      HYUNDAI|  Tucson|  426|
|          BMW|      X5|  351|
+-------------+--------+-----+
only showing top 10 rows



## Different Models

In [ ]:
#Unique models in the dataset
df.select("Model").distinct().show()

+--------------------+
|               Model|
+--------------------+
|           Optima ex|
|                 MDX|
|          Juke Turbo|
|   535 comfort-sport|
|       GL 350 დიზელი|
|              Atenza|
|            7.30E+34|
|     Outlander სპორტ|
|        XC90 3.2 AWD|
|           FIT SPORT|
|             GLA 250|
|               Astra|
|             Mazda 6|
|               Verso|
|Mazda 6 Grand tou...|
|   CLK 320 avangarde|
|            Zafira B|
|          Malibu eco|
|          FIT HIBRID|
|             Korando|
+--------------------+
only showing top 20 rows



In [ ]:
# count of cars with differt fuel types ordered by the Manufacturer name
df.groupBy("Manufacturer","Fuel type").count().orderBy("Manufacturer").show(25)

+------------+--------------+-----+
|Manufacturer|     Fuel type|count|
+------------+--------------+-----+
|       ACURA|        Petrol|   15|
|  ALFA ROMEO|        Petrol|    4|
|ASTON MARTIN|        Petrol|    1|
|        AUDI|        Diesel|   25|
|        AUDI|           CNG|    2|
|        AUDI|        Petrol|  225|
|        AUDI|        Hybrid|    2|
|     BENTLEY|        Petrol|    2|
|         BMW|        Petrol|  715|
|         BMW|           CNG|   33|
|         BMW|        Diesel|  252|
|         BMW|Plug-in Hybrid|    1|
|         BMW|           LPG|   26|
|         BMW|        Hybrid|    9|
|       BUICK|        Petrol|   16|
|    CADILLAC|        Petrol|   12|
|    CADILLAC|           CNG|    1|
|    CADILLAC|           LPG|    1|
|   CHEVROLET|        Diesel|  267|
|   CHEVROLET|        Petrol|  614|
|   CHEVROLET|        Hybrid|  104|
|   CHEVROLET|Plug-in Hybrid|   45|
|   CHEVROLET|           LPG|   17|
|    CHRYSLER|           LPG|    1|
|    CHRYSLER|        Petrol

In [ ]:
#Average price of top 20 car of different Manufacturer ordered by the Average price 
df.groupBy("Manufacturer").agg(avg("Price").alias("Average Price")).orderBy("Average Price",ascending=False).show(20)

+-------------+------------------+
| Manufacturer|     Average Price|
+-------------+------------------+
|  LAMBORGHINI|          872946.0|
|      BENTLEY|          197574.5|
|         OPEL| 73474.89393939394|
|      FERRARI|           66955.5|
|   LAND ROVER|        54807.1875|
| ASTON MARTIN|           54000.0|
|        TESLA|           53941.0|
|      PORSCHE|48406.692307692305|
|       JAGUAR| 34408.78571428572|
|       HUMMER|           31210.6|
|    SSANGYONG|30921.858769931663|
|         JEEP|25409.427536231884|
|      HYUNDAI| 22306.00080450523|
|          BMW|20609.468146718147|
|     MASERATI|           20149.5|
|        LEXUS|20052.594390507013|
|MERCEDES-BENZ|18590.331375428294|
|     INFINITI| 17992.30769230769|
|         MINI| 17481.76595744681|
|         სხვა|           17248.5|
+-------------+------------------+
only showing top 20 rows



In [ ]:
#Price of a car with leather Interior
df.select("Manufacturer","Model","Price").filter("Leather_interior == 'Yes'").show()

+-------------+----------+-----+
| Manufacturer|     Model|Price|
+-------------+----------+-----+
|      HYUNDAI|  Santa FE|36065|
|        HONDA|       FIT|10663|
|       TOYOTA|      Aqua|25669|
|         AUDI|A3 PREMIUM|33243|
|      HYUNDAI|    Sonata|64463|
|      LINCOLN|  Town Car|30420|
|MERCEDES-BENZ|     C 230| 7213|
|        LEXUS|    RX 450| 1019|
|      HYUNDAI|    Sonata|45044|
|       TOYOTA|     RAV 4| 7527|
|       TOYOTA|     Prius| 1882|
|MERCEDES-BENZ|    ML 350|  157|
|        HONDA|   Insight|   94|
|        HONDA|     Civic|56509|
|    CHEVROLET|   Lacetti|12861|
|       NISSAN|     Tiida| 8624|
|    CHEVROLET|     Spark| 3293|
|MERCEDES-BENZ|    ML 320|11917|
|        HONDA|       FIT|10569|
|    SSANGYONG|    Actyon|42171|
+-------------+----------+-----+
only showing top 20 rows



In [ ]:
##number of cars with different fuel types
df.groupBy("Fuel type").count().show()

+--------------+-----+
|     Fuel type|count|
+--------------+-----+
|        Diesel| 4001|
|        Hybrid| 3539|
|      Hydrogen|    1|
|           CNG|  469|
|Plug-in Hybrid|   85|
|           LPG|  885|
|        Petrol| 9944|
+--------------+-----+



In [ ]:
#Information on cars which is being manufacturered by KIA motors and Production year is 2010
df.select('*')\
.filter("Manufacturer == 'KIA'")\
.filter("Prod_year == '2010'")\
.sort("Price",ascending=True)\
.show()

+-----+----+------------+--------+---------+--------+----------------+---------+-------------+---------+-------------+------------+------+----------+------+-------+----------+
|Price|Levy|Manufacturer|   Model|Prod_year|Category|Leather_interior|Fuel type|Engine volume|Cylinders|Gear box type|Drive wheels| Doors|     Wheel| Color|Airbags|Mileage_km|
+-----+----+------------+--------+---------+--------+----------------+---------+-------------+---------+-------------+------------+------+----------+------+-------+----------+
|10506| 683|         KIA|    SOUL|     2010|   Sedan|             Yes|   Petrol|          1.6|      4.0|    Automatic|       Front|04-May|Left wheel| Black|      4|     56864|
|11200| 911|         KIA|Carnival|     2010| Minivan|             Yes|   Diesel|          2.2|      4.0|    Automatic|       Front|04-May|Left wheel|Silver|      4|    125000|
|15367| 911|         KIA| Sorento|     2010|    Jeep|             Yes|   Diesel|          2.2|      4.0|    Automatic|  

In [ ]:
#Average milage of a different Manufacturer different models
df.groupBy("Manufacturer","Model").agg(avg("Mileage_km").alias("Average Mileage")).orderBy("Manufacturer").show(25)

+------------+----------------+------------------+
|Manufacturer|           Model|   Average Mileage|
+------------+----------------+------------------+
|       ACURA|             TSX|         137127.75|
|       ACURA|        TL saber|          205502.0|
|       ACURA|             MDX|151549.66666666666|
|       ACURA|              TL|          221246.5|
|       ACURA|             TLX|           76830.5|
|       ACURA|             RDX|           27200.0|
|  ALFA ROMEO|             147|          240000.0|
|  ALFA ROMEO|       Giulietta|           85000.0|
|  ALFA ROMEO|             166|          200000.0|
|  ALFA ROMEO|             159|          152200.0|
|ASTON MARTIN|          Virage|           72000.0|
|        AUDI|              A5| 86057.66666666667|
|        AUDI|A4 premium plius|          181155.0|
|        AUDI|           A4 B7|          226000.0|
|        AUDI|              A6|       149238.9375|
|        AUDI|              S3|           61410.0|
|        AUDI|           A4 B6|